In [1]:
# imports
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import bz2

In [5]:
# dataset loading
DATASET_PATH_JSON_BZ2 = "../data/raw/quotes-2020.json.bz2"
DATASET_PATH_JSON = "../data/raw/quotes-2020.json"
DATASET_PATH_PICKLE ="../data/binary/data.pickle"

data = [] 
columns = ['quoteID', 'quotation', 'speaker']

def save_pickle(json_path_bz2, pickle_path):
    with bz2.open(json_path_bz2, 'rb') as s_file:
        count = 0
        for instance in tqdm(s_file):
            count += 1
            instance = json.loads(instance) # loading a sample
            if instance['speaker'] == "None":
                continue
            row = dict((k, instance[k]) for k in columns)
            data.append(row)
            if count % 100000 == 0:
                print(count)
        df = pd.DataFrame(data)
        df.to_pickle(pickle_path)

save = False
if save:
    df = save_pickle(DATASET_PATH_JSON_BZ2, DATASET_PATH_PICKLE)

In [6]:
df = pd.read_pickle(DATASET_PATH_PICKLE)
df

,quoteID,quotation,speaker
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick
1,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds
2,2020-01-17-000357,[ The delay ] will have an impact [ on Slough ...,Dexter Smith
3,2020-04-02-000239,[ The scheme ] treats addiction as an illness ...,Barry Coppinger
4,2020-03-19-000276,[ These ] actions will allow households who ha...,Ben Carson
...,...,...,...
3443600,2020-03-03-079268,you're going to take care of the gun problem w...,Joe Biden
3443601,2020-02-24-080186,"you're seeing a young team that's maturing, th...",Brendan Whittet
3443602,2020-02-07-122251,"You're talking about African-Americans, right?...",Barry Michael Cooper
3443603,2020-02-04-118820,You've got to sometimes take that leap of fait...,Brad Gushue


In [7]:
df['speaker'].value_counts()

President Donald Trump    28023
Bernie Sanders            13189
Joe Biden                 12817
President Trump           12415
Andrew Cuomo              11088
                          ...  
Gail Peck                     1
Huy Pham                      1
Prince Johnson                1
Ahmad Zubi                    1
Tjungkara Ken                 1
Name: speaker, Length: 218414, dtype: int64

In [17]:
# extra speaker info
PARQUET_PATH = "../data/raw/speaker_attributes.parquet"
df_meta = pd.read_parquet(PARQUET_PATH)

In [50]:
df_meta.head()
df_meta = df_meta[["aliases", "US_congress_bio_ID", "party", "label"]]

In [51]:
df_meta_notna = df_meta[df_meta['US_congress_bio_ID'].notna() & df_meta['party'].notna()]

In [52]:
df_meta_notna.head()

,aliases,US_congress_bio_ID,party,label
0,"[Washington, President Washington, G. Washingt...",W000178,[Q327591],George Washington
57,"[Elisha Meredith, Elisha Edward Meredith]",M000647,[Q29552],Elisha E. Meredith
79,"[Willard Mitt Romney, Pierre Delecto]",R000615,[Q29468],Mitt Romney
196,"[Richard Milhous Nixon, Nixon, President Nixon...",N000116,[Q29468],Richard Nixon
234,"[President Jefferson, T. Jefferson]",J000069,[Q42186],Thomas Jefferson


In [53]:
df_meta_notna["party"] = df_meta_notna["party"].apply(lambda x: x[0])

C:\Users\mstyc\AppData\Local\Temp/ipykernel_16880/23628598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_notna["party"] = df_meta_notna["party"].apply(lambda x: x[0])


In [54]:
df_meta_notna.head()

,aliases,US_congress_bio_ID,party,label
0,"[Washington, President Washington, G. Washingt...",W000178,Q327591,George Washington
57,"[Elisha Meredith, Elisha Edward Meredith]",M000647,Q29552,Elisha E. Meredith
79,"[Willard Mitt Romney, Pierre Delecto]",R000615,Q29468,Mitt Romney
196,"[Richard Milhous Nixon, Nixon, President Nixon...",N000116,Q29468,Richard Nixon
234,"[President Jefferson, T. Jefferson]",J000069,Q42186,Thomas Jefferson


In [55]:
df_meta_notna["party"].value_counts()

Q29552       5617
Q29468       4541
Q42186        540
Q42183        528
Q42189        323
             ... 
Q19867638       1
Q7054998        1
Q6542163        1
Q1128321        1
Q581259         1
Name: party, Length: 81, dtype: int64

In [56]:
democrats = df_meta_notna[df_meta_notna["party"] == "Q29552"]
republican = df_meta_notna[df_meta_notna["party"] == "Q29468"]

In [61]:
df_meta_notna[df_meta_notna["party"] == "Q42183"].head()

,aliases,US_congress_bio_ID,party,label
235,"[J. Tyler, President Tyler, John Tyler, Jr., J...",T000450,Q42183,John Tyler
5885,None,C000104,Q42183,William B. Campbell
6868,None,M000284,Q42183,Horace Maynard
7063,None,B001058,Q42183,Samuel Bunch
7387,None,R000135,Q42183,David Addison Reese
